# Написать API, который бы по интервалу дат возвращал статистику по часам:

1. Find total amount of orders for each hour
2. Find total count of successful orders for each hour 
3. Find average for last 7 hours
4. Find ratio of orders count to orders count of previous day
5. Find number of unique riders per hour regardless of the state of order
6. Find number of unique drivers per hour regardless of the state of order


In [1]:
import pandas as pd
from pandas import DataFrame
from datetime import datetime

In [2]:
data = pd.read_csv('fake-orders.csv', index_col=['id'])
data.head(5)

,driver,rider,state,time
id,,,,
0,jmorissette@hackett-russel.com,erdmanethelene@gmail.com,FAIL,2016-06-02 04:55:58
1,nkuphal@von.biz,johnstonmilas@kunze.org,FAIL,2016-06-20 21:13:34
2,okunevarenada@cummings-bahringer.com,wolftrevor@gmail.com,SUCCESS,2016-06-17 10:12:33
3,daisha86@buckridge-ankunding.biz,dlakin@auer.com,FAIL,2016-06-18 17:39:47
4,birdieharris@gmail.com,ywillms@runolfsdottir.net,SUCCESS,2016-06-10 13:32:17


In [3]:
#Given
start = '2016-06-02'
stop = '2016-06-20'
    
start_dt = datetime.strptime(start, "%Y-%m-%d")
end_dt = datetime.strptime(stop, "%Y-%m-%d")

#if start and stop dates are the same, then just set the time 23:59:59

if start == stop:
        end_dt = datetime(
        year=end_dt.year, 
        month=end_dt.month,
        day=end_dt.day,
        hour=23,
        minute=59,
        second=59
    )
    
#Sort values
data.time = pd.to_datetime(data.time)
data = data.sort_values(by='time')

#Mask values between start and stop time
mask = (data.time > start_dt) & (data.time <= end_dt)

data_masked = data[mask]
data_masked.head()

,driver,rider,state,time
id,,,,
13900,arutherford@yahoo.com,thompsonmarianne@carter.info,SUCCESS,2016-06-02 00:00:23
18965,bjacobs@klocko-hammes.org,audley07@yahoo.com,SUCCESS,2016-06-02 00:02:09
46105,atharaynor@hammes.com,emardlindsay@tremblay.com,SUCCESS,2016-06-02 00:02:13
17565,atharaynor@hammes.com,ujenkins@barrows.net,FAIL,2016-06-02 00:02:17
618,camronhermiston@ledner.org,doyleizabella@johns-mante.org,SUCCESS,2016-06-02 00:02:47


## 1.Finding total count of orders for each hour

In [4]:
idx_time = data_masked.set_index('time')

result_df = DataFrame(idx_time['state'].resample('1h').count())
result_df = result_df.rename(columns={'state': 'orders_per_hour'})
result_df.head()

,orders_per_hour
time,
2016-06-02 00:00:00,118
2016-06-02 01:00:00,95
2016-06-02 02:00:00,120
2016-06-02 03:00:00,117
2016-06-02 04:00:00,89


## 2.Finding total count of successful orders for each hour 

In [5]:
successful_count = idx_time['state'][idx_time['state'] == 'SUCCESS'].resample('1h').count()

if 'successCount' not in result_df.columns:
    result_df = pd.concat([result_df, successful_count], axis=1)
    result_df = result_df.rename(columns={'state': 'successCount'})
    
result_df.head()

,orders_per_hour,successCount
time,,
2016-06-02 00:00:00,118,95
2016-06-02 01:00:00,95,75
2016-06-02 02:00:00,120,95
2016-06-02 03:00:00,117,99
2016-06-02 04:00:00,89,73


## 3.Finding average for last 7 hours

In [6]:
if 'avg_for_7h' in result_df.columns:
    result_df = result_df.drop('avg_for_7h', axis=1)

result_df['avg_for_7h'] = result_df['orders_per_hour'].rolling('7h').mean().round(2)

result_df.head()


,orders_per_hour,successCount,avg_for_7h
time,,,
2016-06-02 00:00:00,118,95,118.0
2016-06-02 01:00:00,95,75,106.5
2016-06-02 02:00:00,120,95,111.0
2016-06-02 03:00:00,117,99,112.5
2016-06-02 04:00:00,89,73,107.8


## 4. Finding ratio of orders count to orders count of previous day

In [7]:
if 'today_yesterday_ratio' in result_df.columns:
    result_df = result_df.drop('today_yesterday_ratio', axis=1)
    
result_df['today_yesterday_ratio'] = result_df['orders_per_hour'].pct_change(periods=24).round(2)

pd.concat([result_df.iloc[:2], result_df.iloc[24:26]], axis=0)

,orders_per_hour,successCount,avg_for_7h,today_yesterday_ratio
time,,,,
2016-06-02 00:00:00,118,95,118.00,NaN
2016-06-02 01:00:00,95,75,106.50,NaN
2016-06-03 00:00:00,105,89,101.00,-0.11
2016-06-03 01:00:00,97,77,98.57,0.02


## 5. Finding number of unique riders per hour regardless of the state of order

In [8]:
if 'unique_riders_count' in result_df.columns:
    result_df = result_df.drop('avg_for_7h', axis=1)

result_df['unique_riders_count'] = DataFrame(idx_time['rider'].resample('1h').nunique())

result_df.head()

,orders_per_hour,successCount,avg_for_7h,today_yesterday_ratio,unique_riders_count
time,,,,,
2016-06-02 00:00:00,118,95,118.0,NaN,118
2016-06-02 01:00:00,95,75,106.5,NaN,95
2016-06-02 02:00:00,120,95,111.0,NaN,120
2016-06-02 03:00:00,117,99,112.5,NaN,114
2016-06-02 04:00:00,89,73,107.8,NaN,89


## 6. Finding number of unique drivers per hour regardless of the state of order

In [9]:
if 'unique_drivers_count' in result_df.columns:
    result_df = result_df.drop('avg_for_7h', axis=1)

result_df['unique_drivers_count'] = DataFrame(idx_time['driver'].resample('1h').nunique())

result_df.head()

,orders_per_hour,successCount,avg_for_7h,today_yesterday_ratio,unique_riders_count,unique_drivers_count
time,,,,,,
2016-06-02 00:00:00,118,95,118.0,NaN,118,95
2016-06-02 01:00:00,95,75,106.5,NaN,95,84
2016-06-02 02:00:00,120,95,111.0,NaN,120,104
2016-06-02 03:00:00,117,99,112.5,NaN,114,96
2016-06-02 04:00:00,89,73,107.8,NaN,89,79


## 7. Saving result to result.csv

In [10]:
result_df.to_csv('result.csv')